# BaseLine 설명
각 column의 의미는 다음과 같습니다.
- `faultNumber` : 정상인지, 비정상인지 나타내는 Label 입니다. 정상일 경우 `0`, 비정상일 경우 `1` 입니다. 
- `simulationRun` : 시뮬레이션이 실행된 Run 의 번호 입니다. 
  - 동일한 하나의 `simulationRun` 이 정상일 경우 `faultNumber` 가 모두 `0` 입니다.
  - 반대로 하나의 `simulationRun` 이 비정상일 경우 `faultNumber` 가 모두 `1` 입니다.
  - 학습 데이터에는 정상 데이터만 존재합니다. 따라서 `faultNumber` 가 모두 `0` 입니다.
  - 테스트 데이터에는 정상/비정상 데이터가 모두 존재합니다. 따라서 `faultNumber` 가 모두 `0`인 `simulationRun` 도 있고, `faultNumber` 모두 `1`인 `simulationRun` 도 있습니다.
- `sample` : 하나의 Run 안의 sample 번호를 의미합니다. 학습 데이터는 한 Run 당 500 sample 이 있습니다.
- `xmeas_*` : measurement 의 약자로, 화학 공정에서 측정된 센서 값 입니다.
- `xmv_*` : manipulated variable 의 약자로, 화학 공정에서 제어되는 값 입니다.
    - measurement 와 manipulated 관련 설명은 [다음 자료](https://chemicalada.blogspot.com/2016/02/classification-of-variables-in-chemical.html)를 참고하시기 바랍니다.

# 주의사항
되도록 시각화 그림은 출력보다 저장하여 확인하는 것을 추천.... 데이터가 많아 렉이 심함

In [4]:
from typing import Union
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
import plotly.io as pio
from plotly.subplots import make_subplots
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 데이터가 많아 plotly로 시각화시 인터렉션 활성화시 팅김 주의!!!! 특히 test데이터는 더더욱
config = {
    'staticPlot': True,  # 인터랙션 비활성화 (호버, 확대/축소 등)
}



In [5]:
DATA_PATH = Path("C:/Users/lim/Desktop/upstage AI/AD_project/data")
train_data = pd.read_csv(DATA_PATH / "train.csv")
test_data = pd.read_csv(DATA_PATH / "test.csv")

In [6]:
def process_data(df) -> pd.DataFrame:
    numeric_cols = [
        'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6',
        'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12',
        'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18',
        'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24',
        'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30',
        'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36',
        'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1',
        'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8',
        'xmv_9', 'xmv_10', 'xmv_11', 'simulationRun'
    ]
    return df[numeric_cols]

In [7]:
train_a = process_data(train_data)
test_a = process_data(test_data)

In [ ]:

subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig1 = px.line(test_data[["simulationRun"]].iloc[:5000])
fig2 = px.line(test_data[["sample"]].iloc[:5000])
fig2.update_traces(yaxis="y2")

subfig.layout.yaxis.title="index"
subfig.layout.yaxis.title="simulationRun"
subfig.layout.yaxis2.title="sample"
subfig.layout.title = "simulationRun and sample plot" 
subfig.add_traces(fig1.data + fig2.data)
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
pio.show(subfig, config=config)

In [ ]:
train_a.hist(figsize=(20, 15))
plt.tight_layout()
plt.show()

In [ ]:
test_a.hist(figsize=(20, 15))
plt.tight_layout()
plt.show()

In [ ]:
fig = make_subplots(rows= 7, cols= 8)
row, col = 1, 1  # 초기 row와 col 설정

for i, column in enumerate(train_a.columns):
    # 열 데이터에 해당하는 y값으로 그래프 추가
    fig.add_trace(go.Box(y=train_data[column],name=column),
                  row=row, col=col)
    
    # 열의 순서에 따라 col을 증가시키고 8 이상이면 row 증가
    col += 1
    if col > 8:  # 8 열마다 새 행으로 넘어감
        col = 1
        row += 1

# fig.show()를 통해 그래프를 확인
fig.update_layout(
    title='Box plot',
    width=1200, height=1200,
   # staticPlot=True  # 호버 기능 제거 및 정적 렌더링
)

# 그래프 표시
pio.show(fig, config=config)

plotly 그림 저장을 위해 아래 설치 필요

conda install -c conda-forge python-kaleido

설치 안될시 plt 사용

In [12]:
# df => 데이터
def xmv_vs_xmeas_corr(df, type, text):
    if type == 'plotly':
        for i in range(1, 12):  # xmv_1 ~ xmv_11에 대해 반복
            xmv_col = f'xmv_{i}'  # 현재 분석할 xmv 컬럼
            
            # xmv 컬럼과 모든 xmeas 컬럼들 간의 상관관계를 계산
            correlation_series = df[[xmv_col]].join(df[[f'xmeas_{j}' for j in range(1, 42)]]).corr().iloc[0, 1:]

            # Plotly 막대 그래프 생성
            fig = go.Figure(data=[
                go.Bar(
                    x=correlation_series.index,
                    y=correlation_series.values,
                    text=[f'{val:.2f}' for val in correlation_series.values],  # 상관계수 표시
                    textposition='outside',  # 막대 위에 텍스트 위치
                    marker_color='skyblue'  # 막대 색상 설정
                )
            ])

            # 그래프 레이아웃 설정
            fig.update_layout(
                title=f"Correlation: {xmv_col} vs xmeas_1~41",
                xaxis_title="xmeas Variables",
                yaxis_title="Correlation Coefficient",
                xaxis_tickangle=90,  # x축 라벨 회전
                yaxis_range=[-1, 1],  # 상관계수 범위 설정
                height=600, width=800,  # 그래프 크기 조정
                showlegend=False  # 범례 숨기기
            )


            # 이미지로 저장 (파일로 저장 시 `write_image` 사용)
            filename = f"correlation_bar_{xmv_col}_{text}.png"
            fig.write_image(filename, scale=3)
            
    if type == 'plt':
        for i in range(1, 12):  # xmv_1 ~ xmv_10에 대해 반복
            xmv_col = f'xmv_{i}'  # 현재 분석할 xmv 컬럼
            
            # xmv 컬럼과 모든 xmeas 컬럼들 간의 상관관계를 계산
            correlation_series = df[[xmv_col]].join(df[[f'xmeas_{j}' for j in range(1, 42)]]).corr().iloc[0, 1:]

            # 막대그래프 그리기
            plt.figure(figsize=(12, 6))
            bars = plt.bar(correlation_series.index, correlation_series.values, color='skyblue')

            # 막대 위에 상관계수 값 표시
            for bar in bars:
                yval = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval:.2f}', ha='center', va='bottom')

            # 그래프 설정
            plt.title(f"Correlation: {xmv_col} vs xmeas_1~41")
            plt.xlabel('xmeas Variables')
            plt.ylabel('Correlation Coefficient')
            plt.xticks(rotation=90)  # x축 라벨 회전
            plt.ylim(-1, 1)  # 상관계수 범위
            plt.tight_layout()  # 레이아웃 조정

            # 파일로 저장
            filename = f"correlation_bar_{xmv_col}_{text}.png"
            plt.savefig(filename)
            plt.close()  # 저장 후 플롯 닫기


In [13]:
xmv_vs_xmeas_corr(test_a, 'plotly' ,'test')

In [14]:
xmv_vs_xmeas_corr(train_a, 'plotly' ,'train')

In [21]:
def heatmap_xmv_xmeas(df, graph_on_off='off'):
    xmv_columns = [f'xmv_{i}' for i in range(1, 12)]
    xmv_data = df[xmv_columns]

    # 상관관계 행렬 계산
    correlation_matrix = xmv_data.corr()

    # 각 셀에 표시할 텍스트 생성 (소수점 2자리까지)
    text = [[f'{val:.2f}' for val in row] for row in correlation_matrix.values]

    # Plotly 히트맵 생성
    fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        colorscale='RdBu',  # coolwarm과 비슷한 대체 색상
        zmin=-1, zmax=1,
        text=text,  # 셀 내부에 표시할 텍스트
        texttemplate="%{text}",  # 텍스트 포맷 설정
        ))

    # 그래프 레이아웃 설정
    fig.update_layout(
        title="Correlation Matrix: xmv_1 ~ xmv_11",
        xaxis=dict(tickangle=45),  # x축 라벨 회전
        height=600, width=800
    )

    # 그래프 표시 및 저장
    if graph_on_off == 'on':
        pio.show(fig, config=config)
    filename = "correlation_map_xmv.png"
    fig.write_image(filename, scale=3)
    

    xmv_columns = [f'xmeas_{i}' for i in range(1, 42)]
    xmv_data = df[xmv_columns]

    # 상관관계 행렬 계산
    correlation_matrix = xmv_data.corr()

    # 각 셀에 표시할 텍스트 생성 (소수점 2자리까지)
    text = [[f'{val:.2f}' for val in row] for row in correlation_matrix.values]

    # Plotly 히트맵 생성
    fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        colorscale='RdBu',  # coolwarm과 비슷한 대체 색상
        zmin=-1, zmax=1,
        text=text,  # 셀 내부에 표시할 텍스트
        texttemplate="%{text}",  # 텍스트 포맷 설정
    ))

    # 그래프 레이아웃 설정
    fig.update_layout(
        title="Correlation Matrix: xmeas_1 ~ xmeas_41",
        xaxis=dict(tickangle=45),  # x축 라벨 회전
        height=600, width=800
    )

    # 그래프 표시 및 저장
    if graph_on_off == 'on':
        pio.show(fig, config=config)
    filename = "correlation_map_xmeas.png"
    fig.write_image(filename, scale=3)



In [22]:
heatmap_xmv_xmeas(test_a)

In [23]:
heatmap_xmv_xmeas(train_a)

In [30]:
def PCA_test(data, text, graph_on_off='off'):
    X = StandardScaler().fit_transform(data)
    X_PCA = PCA(n_components=2).fit_transform(X)
    df = pd.DataFrame(dict(
        PCA_1 = X_PCA[:, 0],  # PCA의 첫 번째 주성분
        PCA_2 = X_PCA[:, 1]   # PCA의 두 번째 주성분
    ))

    # Plotly Express를 사용하여 2D 산점도 시각화
    #x축은 PCA의 첫 번째 주성분, y축은 두 번째 주성분

    fig = go.Figure(data=go.Scatter(
        x=df["PCA_1"],
        y=df["PCA_2"],
        mode='markers',
        marker=dict(size=8, color='blue', opacity=0.6),
        hoverinfo='skip'  # 호버 정보 비활성화
    ))
        # 레이아웃 설정
    fig.update_layout(
        title="PCA 2D Scatter Plot",
        xaxis_title="PCA 1",
        yaxis_title="PCA 2",
        width=800, height=600
    )

    # 그래프 표시
    if graph_on_off == 'on':
        pio.show(fig, config=config)
    filename = f"PCA_graph_{text}.png"
    fig.write_image(filename, scale=3)

In [31]:
PCA_test(train_a,'train')

In [ ]:
PCA_test(test_a, 'test')